In [ ]:
import sqlite3
import plotly.graph_objs as go
# import plotly.express as px
import plotly.io as pio
import pandas as pd
from run import get_client, get_all_orders

# plotly setup
pio.renderers.default = 'notebook'
pd.options.plotting.backend = 'plotly'


def pwrite(fig, plt='/tmp/vis/plot.json'):
    fig = fig.update_layout(autosize=False)
    fig.write_json(plt)

# KH analysis

In [ ]:
db = sqlite3.connect('/home/n/.data/kh.db')
cur = db.cursor()

In [ ]:
client = get_client('/home/n/.config/square/token')
res = get_all_orders(client)

In [ ]:
len(res)

In [ ]:
# prepare input data for recursive function
inpt = pd.DataFrame({'transactions': res})
inpt['transactions'] = inpt['transactions'].apply(lambda x: [x])

In [ ]:
def convert_unhashable_columns_to_df(df):
    # unhashable columns are lists of more json structure
    # they need to be expanded into their own tables
    output = {}

    # find unhashable columns in input dataframe
    uhc = [c for c in df.columns if df[c].apply(lambda x: isinstance(x, list)).any()]

    for key in uhc:
        # expand tables (normalize the json structures)
        srs = df[key]
        frame = pd.json_normalize(srs[srs.notna()])
        frame = pd.json_normalize(frame.stack())

        # convert date columns to date type
        for d in [c for c in frame.columns if c.endswith('_at')]:
            frame[d] = pd.to_datetime(frame[d], format='ISO8601')

        # recursively check the resulting dataframe for more unhashable columns
        iframes, to_drop = convert_unhashable_columns_to_df(frame)

        if iframes:
            # update `output` with any expanded dataframes
            output.update(iframes)

            # drop extracted keys from current frame
            frame = frame.drop(to_drop, axis=1)

        # add to `output`
        output[key] = frame

    return output, uhc

In [ ]:
frames, _ = convert_unhashable_columns_to_df(inpt)

In [ ]:
for name, frame in frames.items():
    frame.columns = [c.replace('.', '_') for c in frame.columns]
    frame.to_sql(name, db, if_exists='replace')

In [ ]:
df = frames['transactions']
df['total_money_amount'] *= 0.01
df['date'] = df['closed_at'].dt.date
df['day'] = df['closed_at'].dt.day
df['week'] = df['closed_at'].dt.isocalendar().week
df['month'] = df['closed_at'].dt.month
df['year'] = df['closed_at'].dt.year

In [ ]:
df.info()

In [ ]:
tp = df.groupby(['year', 'week']).agg({
    'date': 'min',
    'id': 'count',
    'total_money_amount': 'sum',
})

In [ ]:
tp

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tp['date'],
    y=tp['total_money_amount'],
))

pwrite(fig)